In [1]:
import pandas as pd

file_path = 'postings.csv'
df_or = pd.read_csv(file_path)

df = df_or[['job_id','company_name', 'title','description','skills_desc','normalized_salary','formatted_experience_level','formatted_work_type','remote_allowed','posting_domain','location','listed_time','zip_code']]
# print(df.head())

In [ ]:
df = df.fillna({
    'company_name': 'Unknown',
    'description': '',
    'skills_desc': '',
    'normalized_salary': 0,
    'remote_allowed': False,
    'zip_code': 'Unknown'
})
df.loc[df['remote_allowed']=='1.0','remote_allowed']=True

In [3]:
# Base dataset (for NLP + analysis)
df_base = df.dropna(subset=['job_id','title','description'])
df_base = df_base.drop_duplicates(subset='job_id')

# Salary dataset (for prediction)
df_salary = df_base.copy()
df_salary['normalized_salary'] = pd.to_numeric(df_salary['normalized_salary'], errors='coerce')
df_salary = df_salary.dropna(subset=['normalized_salary'])
df_salary = df_salary[df_salary['normalized_salary'] > 0]


In [4]:
df_base.to_csv('cleaned_postings.csv', index=False)

In [5]:
SKILL_LIST = [
    
    # Programming Languages
    "python", "java", "c", "c++", "c#", "javascript", "typescript",
    "go", "golang", "rust", "scala", "kotlin", "swift", "ruby", "php",
    "r", "matlab", "bash", "shell", "powershell", "sql",

    # Data Analysis / Scientific Python
    "pandas", "numpy", "scipy", "statsmodels", "jupyter", "jupyter notebook",
    "matplotlib", "plotly",

    # Databases & Storage
    "postgresql", "postgres", "mysql", "mariadb", "sqlite",
    "mssql", "sql server", "oracle",
    "mongodb", "cassandra", "dynamodb", "cosmos db",
    "redis", "elasticsearch", "opensearch", "neo4j",
    "snowflake", "redshift", "bigquery", "synapse", "teradata",
    "s3", "adls", "gcs",

    # File formats / table formats
    "parquet", "avro", "orc",
    "delta lake", "delta", "iceberg", "hudi",

    # Big Data / Streaming / Messaging
    "apache spark", "spark", "pyspark",
    "hadoop", "hdfs", "yarn",
    "hive", "presto", "trino",
    "kafka", "kinesis", "pubsub", "google pubsub", "pub/sub",
    "flink", "spark streaming", "storm",
    "rabbitmq", "sqs", "sns",

    # Orchestration / ETL / ELT
    "etl", "elt", "data pipeline", "data pipelines",
    "airflow", "dagster", "prefect", "luigi",
    "dbt", "fivetran", "stitch",
    "informatica", "talend", "ssis",

    # Cloud Platforms & Services
    "aws", "amazon web services", "azure", "gcp", "google cloud",
    # AWS specifics
    "lambda", "api gateway", "ecs", "ecr", "eks", "fargate",
    "sagemaker", "cloudwatch", "cloudformation", "athena", "glue",
    # Azure specifics
    "azure functions", "aks", "acr", "azure devops", "data factory",
    # GCP specifics
    "cloud functions", "cloud run", "gke", "bigtable", "dataflow",


    # Containers / Infra / DevOps
    "docker", "kubernetes", "helm",
    "terraform", "pulumi", "ansible",
    "jenkins", "github actions", "gitlab ci", "circleci", "ci/cd",
    "nginx", "apache",

    # Backend / APIs
    "rest", "rest api", "graphql", "grpc", "soap",
    "fastapi", "flask", "django",
    "node.js", "nodejs", "express",
    "spring", "spring boot", ".net", "asp.net", "asp.net core",

    # Frontend (common in full-stack postings)
    "react", "next.js", "nextjs", "vue", "angular",
    "html", "css", "sass", "tailwind", "bootstrap",

    # Machine Learning / AI Fundamentals
    "machine learning", "deep learning", "artificial intelligence",
    "statistics", "time series", "forecasting",
    "natural language processing", "nlp",
    "computer vision", "recommendation systems", "recommender systems",
    "a/b testing", "ab testing", "experiment design",

    # ML models / methods
    "linear regression", "logistic regression",
    "random forest", "xgboost", "lightgbm", "catboost",
    "svm", "naive bayes", "k-means", "clustering",
    "neural networks",

    # ML / AI Frameworks
    "scikit-learn", "sklearn",
    "tensorflow", "keras", "pytorch",
    "xgboost", "lightgbm",  # (kept here too because postings repeat)
    "hugging face", "transformers",
    "spacy", "nltk",
    "opencv",
    "mlflow", "kubeflow",

    # GenAI / LLM (very common now)
    "llm", "large language model", "generative ai", "genai",
    "prompt engineering", "rag", "retrieval augmented generation",
    "vector database", "vector db", "embeddings",
    "langchain", "llamaindex",
    "openai", "azure openai",
    "pinecone", "weaviate", "milvus", "faiss", "chromadb", "chroma",

    # MLOps / Deployment / Serving
    "mlops", "model deployment", "model serving", "model monitoring",
    "feature store",
    "seldon", "bentoml", "ray serve",
    "onnx",

    # Observability / Monitoring / Logging
    "prometheus", "grafana",
    "datadog", "new relic",
    "elk", "splunk",

    # Security (often in job reqs)
    "oauth", "oauth2", "openid connect", "jwt",
    "iam", "rbac",
    "encryption", "tls", "ssl",

    # Testing / Quality
    "unit testing", "integration testing", "e2e testing",
    "pytest", "unittest", "junit",
    "selenium", "cypress",

    # Version Control / Collaboration
    "git", "github", "gitlab", "bitbucket",
    "agile", "scrum", "jira", "confluence",

    # BI / Analytics Tools
    "excel", "power bi", "tableau", "looker", "metabase", "superset",

    # Data Modeling / Warehousing Concepts
    "data modeling", "dimensional modeling", "star schema", "snowflake schema",
    "data warehousing", "data lake", "lakehouse"
]


In [6]:
from spacy.lang.en import English
from spacy.matcher import PhraseMatcher

nlp = English()
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
patterns = [nlp.make_doc(name) for name in SKILL_LIST]
matcher.add("Skills", patterns)
desc = df_base['description'].fillna("").astype(str)
skills = df_base['skills_desc'].fillna("").astype(str)

desc_skill = (desc + " " + skills).str.strip().tolist()
batch = 256
result = []
for i, doc in enumerate(nlp.pipe(desc_skill,batch_size=batch)):
    matches = matcher(doc)
    skills_found = set()
    for match_id, start, end in matches:
        span = doc[start:end]
        skills_found.add(span.text.lower())
    result.append("|".join(sorted(skills_found)))

df_base['extracted_skills'] = result



In [10]:
# df_base.to_csv('cleaned_postings_with_skills.csv', index=False)
df_base.to_parquet('cleaned_postings.parquet',engine='pyarrow', compression='snappy',index=False)

ArrowInvalid: ('Could not convert 1.0 with type float: tried to convert to boolean', 'Conversion failed for column remote_allowed with type object')